In [1]:
"""
Extract Active Fire Detections (AFDs) from MODIS and VIIRS within "aspen fires"

Author: maxwell.cook@colorado.edu
"""

import os, time, glob, sys
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore") # suppresses annoying geopandas warning

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

proj = 'EPSG:5070'

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/Aim2/')

print("Ready to go !")

Ready to go !


In [2]:
# Load the fire data.

In [3]:
aspen_events_fp = os.path.join(projdir,'data/spatial/mod/NIFC/nifc-ics_2018_to_2023-aspen.gpkg')
aspen_events = gpd.read_file(aspen_events_fp)
aspen_events.columns

Index(['NIFC_ID', 'NIFC_NAME', 'NIFC_ACRES', 'FINAL_ACRES', 'pct_aspen',
       'INCIDENT_ID', 'INCIDENT_NAME', 'START_YEAR', 'CAUSE', 'DISCOVERY_DATE',
       'DISCOVERY_DOY', 'WF_CESSATION_DATE', 'WF_CESSATION_DOY',
       'STR_DESTROYED_TOTAL', 'STR_DAMAGED_TOTAL', 'STR_THREATENED_MAX',
       'EVACUATION_REPORTED', 'PEAK_EVACUATIONS', 'WF_PEAK_AERIAL',
       'WF_PEAK_PERSONNEL', 'na_l3name', 'geometry'],
      dtype='object')

In [4]:
# Create a fire perimeter buffer to extract FRP observations within 1km of bounds.
bounds = create_bounds(aspen_events, by_bounds=True, buffer=1000)
bounds = bounds.set_crs(proj, allow_override=True) # ensure correct crs
bounds = bounds[['NIFC_ID','NIFC_NAME','START_YEAR','DISCOVERY_DATE','WF_CESSATION_DATE','na_l3name','geometry']] 
bounds['START_YEAR'] = bounds['START_YEAR'].astype(int)
date_cols = ['DISCOVERY_DATE', 'WF_CESSATION_DATE']
for col in date_cols:
    bounds[col] = pd.to_datetime(bounds[col], errors='coerce')
bounds.head()

,NIFC_ID,NIFC_NAME,START_YEAR,DISCOVERY_DATE,WF_CESSATION_DATE,na_l3name,geometry
0,6,DEVIL CREEK,2018,2018-07-19 15:46:00,2018-07-21 15:00:00,Southern Rockies,"POLYGON ((-981360.461 1644805.834, -981864.344..."
1,8,577,2019,2019-07-28 14:22:00,2019-08-18 16:00:00,Southern Rockies,"POLYGON ((-1003641.705 1627304.106, -1004580.3..."
2,14,416,2018,2018-06-01 11:02:00,2018-07-03 18:00:00,Southern Rockies,"POLYGON ((-1031771.042 1658048.676, -1047969.7..."
3,23,LOADING PEN,2020,2020-06-13 21:42:00,2020-06-18 17:45:00,Southern Rockies,"POLYGON ((-1071014.573 1681756.433, -1071765.1..."
4,24,PLATEAU,2018,2018-07-22 16:18:00,2018-08-17 18:45:00,Southern Rockies,"POLYGON ((-1080396.374 1682569.954, -1094464.3..."


In [5]:
bounds.crs

<Projected CRS: EPSG:5070>
Name: NAD83 / Conus Albers
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: United States (USA) - CONUS onshore - Alabama; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming.
- bounds: (-124.79, 24.41, -66.91, 49.38)
Coordinate Operation:
- name: Conus Albers
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [6]:
# Load the Active Fire Detection (AFD) data

AFDs from MODIS Collection 6.1 (1km) and the Suomi National Polar-Orbiting Partnership (VIIRS S-NPP 375m) were obtained from the NASA FIRMS (https://firms.modaps.eosdis.nasa.gov/download/) between 2018-2023 in the western US. The S-NPP AFDs include an archive shapefile and a NRT shapefile for Sept 2022-->

In [7]:
# Gather the archive and NRT S-NPP AFDs.
snpp_fp = os.path.join(projdir,'data/spatial/raw/NASA-FIRMS/DL_FIRE_SV-C2_476784/')
vects = glob.glob(snpp_fp+"*.shp")
print([os.path.basename(v) for v in vects])

['fire_nrt_SV-C2_476784.shp', 'fire_archive_SV-C2_476784.shp']


In [8]:
# Merge the NRT and archive vintages
archive = gpd.read_file([v for v in vects if "archive" in v][0]).to_crs(proj)
nrt = gpd.read_file([v for v in vects if "nrt" in v][0]).to_crs(proj)
snpp = pd.concat([archive, nrt], ignore_index=True)
snpp.head()

,LATITUDE,LONGITUDE,BRIGHTNESS,SCAN,TRACK,ACQ_DATE,ACQ_TIME,SATELLITE,INSTRUMENT,CONFIDENCE,VERSION,BRIGHT_T31,FRP,DAYNIGHT,TYPE,geometry
0,36.622398,-119.958420,319.25,0.45,0.63,2018-01-01,0854,N,VIIRS,n,1,276.46,1.93,N,0.0,POINT (-2100437.814 1774627.392)
1,48.166241,-102.698212,323.92,0.38,0.36,2018-01-01,0854,N,VIIRS,n,1,244.01,3.01,N,0.0,POINT (-502299.189 2814197.682)
2,48.152390,-102.697792,325.34,0.38,0.36,2018-01-01,0854,N,VIIRS,n,1,247.16,2.07,N,0.0,POINT (-502375.241 2812672.753)
3,47.879730,-102.706543,320.74,0.38,0.36,2018-01-01,0854,N,VIIRS,n,1,245.34,3.00,N,0.0,POINT (-505150.481 2782728.231)
4,44.249660,-104.516678,327.49,0.38,0.36,2018-01-01,0854,N,VIIRS,n,1,251.79,5.84,N,0.0,POINT (-677212.174 2392519.636)


In [9]:
# Set up a dictionary to store both MODIS and VIIRS AFDs
modis_fp = os.path.join(projdir,'data/spatial/raw/NASA-FIRMS/DL_FIRE_M-C61_476781/')

# Store these in a dictionary
afds = {
    "MOD61": gpd.read_file(modis_fp).to_crs(proj),
    "SNPP": snpp
}

afds['MOD61'].head()

,LATITUDE,LONGITUDE,BRIGHTNESS,SCAN,TRACK,ACQ_DATE,ACQ_TIME,SATELLITE,INSTRUMENT,CONFIDENCE,VERSION,BRIGHT_T31,FRP,DAYNIGHT,TYPE,geometry
0,44.3762,-119.1190,312.5,1.2,1.1,2018-01-01,0608,Terra,MODIS,85,6.03,269.1,21.9,N,0,POINT (-1819292.701 2598646.798)
1,32.1679,-107.7655,322.3,1.1,1.0,2018-01-01,1809,Terra,MODIS,77,6.03,289.1,19.0,D,0,POINT (-1101321.656 1078752.715)
2,32.1643,-107.7433,318.4,1.1,1.0,2018-01-01,1809,Terra,MODIS,74,6.03,287.9,14.9,D,0,POINT (-1099303.652 1078097.526)
3,31.7188,-102.0085,303.8,1.4,1.2,2018-01-01,1944,Aqua,MODIS,57,6.03,279.4,12.6,D,0,POINT (-566656.554 978378.293)
4,38.8238,-122.6829,302.0,2.7,1.6,2018-01-01,1946,Terra,MODIS,50,6.03,280.2,26.9,D,0,POINT (-2264952.497 2075236.763)


In [10]:
afds['MOD61'].crs

<Projected CRS: EPSG:5070>
Name: NAD83 / Conus Albers
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: United States (USA) - CONUS onshore - Alabama; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming.
- bounds: (-124.79, 24.41, -66.91, 49.38)
Coordinate Operation:
- name: Conus Albers
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [11]:
# Extract the AFDs within fire bounds.

In [12]:
afd_events = {} # to store the filtered data
# Loop through sensors (MODIS and VIIRS)
for satellite, afd in afds.items():
    print(f"Processing: {satellite}")
    
    afd['VID'] = afd.index # add a unique ID
    
    # Remove low confidence observations
    try:
        afd = afd[afd['CONFIDENCE'] != 'l']
    except KeyError as e:
        print(f"KeyError: {e}")

    if afd.crs != bounds.crs:
        afd = afd.to_crs(bounds.crs)  # ensure the crs matches

    # Extract within fire bounds
    afd_ = gpd.sjoin(afd, bounds, how='inner', predicate='within')
    afd_.drop(columns=['index_right'], inplace=True)
    
    # Tidy the date columns for matching
    afd_['ACQ_DATE'] = pd.to_datetime(afd_['ACQ_DATE'])
    afd_['ACQ_MONTH'] = afd_['ACQ_DATE'].dt.month.astype(int)
    afd_['ACQ_YEAR'] = afd_['ACQ_DATE'].dt.year.astype(int)
    
    # Create a datetime object
    afd_.loc[:, 'ACQ_DATETIME'] = afd_.apply(lambda row: convert_datetime(row['ACQ_DATE'], row['ACQ_TIME']), axis=1)
    
    # Filter for matching dates
    afd_f = afd_[
        (afd_['ACQ_YEAR'] >= afd_['DISCOVERY_DATE'].dt.year.astype(int)) & 
        (afd_['ACQ_MONTH'] >= afd_['DISCOVERY_DATE'].dt.month.astype(int)) &
        (afd_['ACQ_YEAR'] <= afd_['WF_CESSATION_DATE'].dt.year.astype(int)) &
        (afd_['ACQ_MONTH'] <= afd_['WF_CESSATION_DATE'].dt.month.astype(int))
    ]

    # Drop any duplicates
    dups = afd_f.duplicated(subset='VID').sum()
    if dups > 0:
        print(f"Be aware ! Duplicates found ! {dups} to be exact ...")
        # afd_f = afd_f.drop_duplicates(subset='VID', keep='first')
    
    print(f"\tThere are [{len(afd_f)}] high or nominal confidence detections within fire bounds.")

    # Overwrite the dictionary items
    afd_events[satellite] = afd_f

    # Save the AFD observation in fire bounds
    out_fp = os.path.join(projdir,f'data/spatial/mod/AFD/{satellite}-afd_aspen-fires_2018_to_2023.gpkg')
    afd_f.to_file(out_fp)
    print(f'Saved to {out_fp}\n')
    
    del afd, afd_, afd_f
    gc.collect()

print("\nProcessing complete !")

Processing: MOD61
Be aware ! Duplicates found ! 904 to be exact ...
	There are [29084] high or nominal confidence detections within fire bounds.
Saved to /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/mod/AFD/MOD61-afd_aspen-fires_2018_to_2023.gpkg

Processing: SNPP
Be aware ! Duplicates found ! 3070 to be exact ...
	There are [120842] high or nominal confidence detections within fire bounds.
Saved to /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/mod/AFD/SNPP-afd_aspen-fires_2018_to_2023.gpkg


Processing complete !


In [13]:
# Handle duplicates seperately

In [14]:
afd_events['MOD61'].columns

Index(['LATITUDE', 'LONGITUDE', 'BRIGHTNESS', 'SCAN', 'TRACK', 'ACQ_DATE',
       'ACQ_TIME', 'SATELLITE', 'INSTRUMENT', 'CONFIDENCE', 'VERSION',
       'BRIGHT_T31', 'FRP', 'DAYNIGHT', 'TYPE', 'geometry', 'VID', 'NIFC_ID',
       'NIFC_NAME', 'START_YEAR', 'DISCOVERY_DATE', 'WF_CESSATION_DATE',
       'na_l3name', 'ACQ_MONTH', 'ACQ_YEAR', 'ACQ_DATETIME'],
      dtype='object')

In [15]:
# Create the AFD "detection buffer" for both MODIS and VIIRS

In [16]:
afd_events_plots = {} # to store the buffered observations
for satellite, afd in afd_events.items():
    print(f"Creating buffer for {satellite} observations.")

    afd_ = afd.copy()
    if satellite == 'MOD61':
        afd_.geometry = afd_.geometry.buffer(500, cap_style=3) # square buffer half of pixel size
    elif satellite == 'SNPP':
        afd_.geometry = afd_.geometry.buffer(187.5, cap_style=3) # square buffer half of pixel size

    afd_events_plots[satellite] = afd_ # add to the new dictionary

    # Save out
    out_fp = os.path.join(projdir,f'data/spatial/mod/AFD/{satellite}-afd_aspen-fires_2018_to_2023_buffer.gpkg')
    afd_.to_file(out_fp)
    print(f'Saved to {out_fp}\n')

    del afd_
    gc.collect()

Creating buffer for MOD61 observations.
Saved to /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/mod/AFD/MOD61-afd_aspen-fires_2018_to_2023_buffer.gpkg

Creating buffer for SNPP observations.
Saved to /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/mod/AFD/SNPP-afd_aspen-fires_2018_to_2023_buffer.gpkg



In [17]:
afd_events_plots['MOD61'].columns

Index(['LATITUDE', 'LONGITUDE', 'BRIGHTNESS', 'SCAN', 'TRACK', 'ACQ_DATE',
       'ACQ_TIME', 'SATELLITE', 'INSTRUMENT', 'CONFIDENCE', 'VERSION',
       'BRIGHT_T31', 'FRP', 'DAYNIGHT', 'TYPE', 'geometry', 'VID', 'NIFC_ID',
       'NIFC_NAME', 'START_YEAR', 'DISCOVERY_DATE', 'WF_CESSATION_DATE',
       'na_l3name', 'ACQ_MONTH', 'ACQ_YEAR', 'ACQ_DATETIME'],
      dtype='object')

In [18]:
# Gather bounds for individual fires/AFDs

In [19]:
buffer = 1000 # meters
afd_fire_bounds = {}
for satellite, afd in afd_events_plots.items():
    print(f"Creating bounds for {satellite} observations.")
    
    # Group by fire ID
    grouped = afd.groupby('NIFC_ID')
    bounds_gdf = gpd.GeoDataFrame(columns=['NIFC_ID', 'geometry'], crs=afd.crs)
    # Iterate over each fire group
    for fire_id, group in grouped:
        # Calculate the total bounds (minx, miny, maxx, maxy) for all geometries in the group
        bounds = box(*group.total_bounds)
        # Apply buffer if specified
        if buffer is not None:
            bounds = bounds.buffer(buffer)
        # Create a GeoDataFrame for this fire with the bounding box, id, and ig_year
        fire_gdf = gpd.GeoDataFrame({'NIFC_ID': [fire_id], 'geometry': [bounds]}, crs=afd.crs)
        # Append the result to the GeoDataFrame
        bounds_gdf = pd.concat([bounds_gdf, fire_gdf], ignore_index=True)

    # Store the result
    afd_fire_bounds[satellite] = bounds_gdf

    # Save out the bounds to a file
    out_fp = os.path.join(projdir, f'data/spatial/mod/AFD/{satellite}-afd_aspen-fires_2018_to_2023_bounds.gpkg')
    bounds_gdf.to_file(out_fp)
    print(f"Saved to {out_fp}\n")

    gc.collect()

Creating bounds for MOD61 observations.
Saved to /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/mod/AFD/MOD61-afd_aspen-fires_2018_to_2023_bounds.gpkg

Creating bounds for SNPP observations.
Saved to /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/mod/AFD/SNPP-afd_aspen-fires_2018_to_2023_bounds.gpkg



In [20]:
afd_fire_bounds['MOD61'].head()

,NIFC_ID,geometry
0,100763,"POLYGON ((-1302777.051 2935597.721, -1304470.8..."
1,1055,"POLYGON ((-1662672.883 2765372.027, -1665727.3..."
2,1070,"POLYGON ((-1648256.013 2755617.377, -1673173.6..."
3,12229,"POLYGON ((-1978966.285 1675019.040, -1982045.2..."
4,13137,"POLYGON ((-1587662.179 2645713.160, -1590498.6..."


In [21]:
# Combine the MODIS and VIIRS bounds into a single GeoDataFrame
combined = pd.concat([afd_fire_bounds['MOD61'], afd_fire_bounds['SNPP']], ignore_index=True)
# Dissolve the geometries by fire ID ('OBJECTID') to combine geometries for each fire
combined = combined.dissolve(by='NIFC_ID')
combined = pd.merge(combined, aspen_events.drop(columns=['geometry']), left_on='NIFC_ID', right_on='NIFC_ID', how='left')
combined.columns

Index(['NIFC_ID', 'geometry', 'NIFC_NAME', 'NIFC_ACRES', 'FINAL_ACRES',
       'pct_aspen', 'INCIDENT_ID', 'INCIDENT_NAME', 'START_YEAR', 'CAUSE',
       'DISCOVERY_DATE', 'DISCOVERY_DOY', 'WF_CESSATION_DATE',
       'WF_CESSATION_DOY', 'STR_DESTROYED_TOTAL', 'STR_DAMAGED_TOTAL',
       'STR_THREATENED_MAX', 'EVACUATION_REPORTED', 'PEAK_EVACUATIONS',
       'WF_PEAK_AERIAL', 'WF_PEAK_PERSONNEL', 'na_l3name'],
      dtype='object')

In [22]:
len(combined)

184

In [ ]:
# Join to state boundaries, assign "Start_Day" and "End_Day" from Parks et al. (2019)

In [23]:
states = os.path.join(maindir, 'data/boundaries/political/TIGER/tl19_us_states_w_ak_lambert.gpkg')
states = gpd.read_file(states)
states = states.to_crs(proj)
states.columns

Index(['REGION', 'DIVISION', 'STATEFP', 'STATENS', 'GEOID', 'STUSPS', 'NAME',
       'LSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON',
       'geometry'],
      dtype='object')

In [34]:
centroid = combined.copy()
centroid['geometry'] = centroid.geometry.centroid
fire_state = gpd.overlay(centroid[['NIFC_ID','geometry']], states[['STUSPS','geometry']], how='intersection')
fire_state = fire_state[['NIFC_ID','STUSPS']]
combined_ = pd.merge(combined, fire_state, on='NIFC_ID', how='left')
combined_.columns

Index(['NIFC_ID', 'geometry', 'NIFC_NAME', 'NIFC_ACRES', 'FINAL_ACRES',
       'pct_aspen', 'INCIDENT_ID', 'INCIDENT_NAME', 'START_YEAR', 'CAUSE',
       'DISCOVERY_DATE', 'DISCOVERY_DOY', 'WF_CESSATION_DATE',
       'WF_CESSATION_DOY', 'STR_DESTROYED_TOTAL', 'STR_DAMAGED_TOTAL',
       'STR_THREATENED_MAX', 'EVACUATION_REPORTED', 'PEAK_EVACUATIONS',
       'WF_PEAK_AERIAL', 'WF_PEAK_PERSONNEL', 'na_l3name', 'STUSPS'],
      dtype='object')

In [35]:
# Assign the Parks et al. (2019) start and end days
special_case = ['Arizona', 'New Mexico']
combined_['Start_Day'] = None
combined_['End_Day'] = None    

combined_.loc[combined_['STUSPS'].isin(special_case), ['Start_Day', 'End_Day']] = (91, 181)
combined_.loc[~combined_['STUSPS'].isin(special_case), ['Start_Day', 'End_Day']] = (152, 258)

combined_ = combined_[['NIFC_ID','NIFC_NAME','START_YEAR','Start_Day','End_Day','geometry']]
combined_ = combined_.rename(columns={
    'NIFC_ID': 'Fire_ID', 
    'NIFC_NAME': 'Fire_Name',
    'START_YEAR': 'Fire_Year'
})
combined_.head()

,Fire_ID,Fire_Name,Fire_Year,Start_Day,End_Day,geometry
0,100763,PARK CREEK,2022,152,258,"POLYGON ((-1304470.897 2935597.721, -1304568.9..."
1,1055,RATTLESNAKE,2020,152,258,"POLYGON ((-1665727.343 2765372.027, -1665825.3..."
2,1070,GREEN RIDGE,2021,152,258,"POLYGON ((-1673173.683 2755617.377, -1673271.7..."
3,12229,SCHAEFFER,2019,152,258,"POLYGON ((-1977949.487 1678806.442, -1977925.6..."
4,13137,COUGAR,2020,152,258,"POLYGON ((-1590498.604 2645713.160, -1590596.6..."


In [36]:
# Save out the bounds to a file for GEE.
out_fp = os.path.join(projdir, f'data/earth-engine/imports/afd_aspen-fires_2018_to_2023_bounds.shp')
combined_.to_file(out_fp)
print(f"Saved to {out_fp}\n")

Saved to /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/earth-engine/imports/afd_aspen-fires_2018_to_2023_bounds.shp



In [37]:
gc.collect()

1056